# Modelos Predictivos: Rentas Cedidas Municipales
# SARIMAX, Prophet, XGBoost y Deep Learning

---

## 📋 Proyecto de Grado
**Título**: Predicción del Comportamiento de las Rentas Cedidas en el Financiamiento del Régimen Subsidiado de Salud a Nivel Municipal mediante Modelos de Machine Learning

**Autores**:
- Mauricio García Mojica
- Efrén Bohorquez Vargas
- Ernesto Sánchez García

**Objetivo**: Desarrollar modelos predictivos para estimar ingresos por Rentas Cedidas (2020-2024) y fortalecer la planeación financiera territorial de la ADRES.

### Modelos Implementados:
1. 📊 **SARIMAX**: Modelo econométrico base (Benchmark)
2. 🔮 **Prophet**: Descomposición aditiva con regresores
3. 🌳 **XGBoost**: Gradient Boosting con features de ingeniería
4. 🧠 **LSTM**: Redes Neuronales Recurrentes (Deep Learning)
5. 🎯 **Ensemble**: Combinación ponderada de modelos

---

**Fecha**: Febrero 2026  
**Versión**: 2.0 (Alineada con Registro de Grado)

## 1. Configuración del Entorno

In [ ]:
# Importaciones generales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

# Modelos Econométricos
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Modelos ML y Prophet
from prophet import Prophet
from neuralprophet import NeuralProphet
import xgboost as xgb

# Deep Learning
import torch
import torch.nn as nn

# Optimización y Validación
import optuna
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Entorno configurado para modelos SARIMAX, Prophet, XGBoost y LSTM")

## 2. Carga de Datos (2020-2024)

In [ ]:
# Cargar datasets preparados
print("📁 Cargando datos históricos (2020-2024)...")

# TODO: Cargar archivos reales
# train_df = pd.read_parquet('../data/processed/train_mensual.parquet')
# test_df = pd.read_parquet('../data/processed/test_mensual.parquet')

print("⏳ Pendiente de ejecución con datos depurados")

---
# MODELADO PREDICTIVO
---

## 3. Modelo Base: SARIMAX (Benchmark)

In [ ]:
def entrenar_sarimax(train_series, order=(1,1,1), seasonal_order=(1,1,1,12), exog=None):
    """
    Entrena modelo SARIMAX (Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors)
    """
    print("📊 Entrenando SARIMAX...")
    
    try:
        model = SARIMAX(
            train_series,
            order=order,
            seasonal_order=seasonal_order,
            exog=exog,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        
        results = model.fit(disp=False)
        print("✅ SARIMAX convergencia exitosa")
        print(results.summary().tables[1])
        return results
        
    except Exception as e:
        print(f"❌ Error en SARIMAX: {str(e)}")
        return None

print("✅ Función SARIMAX definida")

## 4. Modelo Prophet (Meta AI)

In [ ]:
def entrenar_prophet(train_df, features_exogenas):
    """
    Entrena Prophet con regresores
    """
    print("🔮 Entrenando Prophet...")
    
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='multiplicative'
    )
    
    for regressor in features_exogenas:
        model.add_regressor(regressor)
    
    # model.fit(train_df)
    print("✅ Prophet configurado")
    return model

print("✅ Función Prophet definida")

## 5. Modelo XGBoost (Machine Learning)

In [ ]:
def entrenar_xgboost(X_train, y_train):
    """
    Entrena XGBoost Regressor
    """
    print("🌳 Entrenando XGBoost...")
    
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        early_stopping_rounds=50
    )
    
    # model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    print("✅ XGBoost configurado")
    return model

print("✅ Función XGBoost definida")

## 6. Modelo LSTM (Deep Learning)

In [ ]:
class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

print("🧠 Arquitectura LSTM definida")

## 7. Comparación y Ensemble

In [ ]:
def evaluar_modelos(predicciones_dict, y_true):
    """
    Compara métricas (RMSE, MAE, MAPE) de todos los modelos
    """
    resultados = []
    
    for nombre, y_pred in predicciones_dict.items():
        mape = mean_absolute_percentage_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        
        resultados.append({
            'Modelo': nombre,
            'MAPE': mape,
            'RMSE': rmse
        })
    
    return pd.DataFrame(resultados).sort_values('MAPE')

print("✅ Función de evaluación definida")

## 8. Ejecución y Resultados

In [ ]:
print("="*80)
print("EJECUCIÓN DEL PIPELINE PREDICTIVO")
print("="*80)

print("\n1. Entrenar SARIMAX (Benchmark)")
print("2. Entrenar Prophet y XGBoost (ML)")
print("3. Entrenar LSTM (Deep Learning)")
print("4. Generar Ensemble ponderado")
print("5. Comparar MAYE y seleccionar mejor modelo")

print("\n⚠️ IMPORTANTE: Ajustar fechas de corte según datos disponibles (2020-2024)")